# Simulation model
<img src ="./quartz_crystal_model.png" width=700 />

In [17]:
# Calculating the motional resistance Rm

ESR = 50 # Equivalent series resistance of the crystal
C0 = 1e-6 # Shunt capacitance of the crystal
C1 = 10e-12 # First external load capacitance of the crystal
C2 = 10e-12 # Second external load capacitance of the crystal

CL = C1 * C2 / (C1 + C2) # Total load capacitance of the crystal

Rm = ESR /  (1 + C0 / CL)**2

## Resonance frequency
First resonance is series frequency of Lm and Cm.
$$f_{s}=\frac{1}{2*\Pi*\sqrt{L_{m}*C_{m}}}$$
Second resonance is parallel frequency of Lm and (Cm||C0).
$$f_{p}=\frac{1}{2*\Pi*\sqrt{L_{m}*\frac{C_{m}*C_{0}}{C_{m}+C_{0}}}}$$

Output of amplifier with external capacitor forms RC filter.  Crystal with the other external capacitor forms another RC filter.  The two RC filters form a 180 degree phase shift at the resonance frequency.

# Pierce oscillator
<img src="./pierce_oscillator_circuit.png" width=700/>

# Stability
## Negative resistance testing
1. Measure Rm of the crystal with a VNA.  It's the resistance at the series resonance frequency. [link](https://youtu.be/MWQCZGXpSSc?list=PLT84nve2j1g9IW7cd41XPkfszVWN84zZv&t=439)
2. Calculate equivalent series resistance (R<sub>e</sub>) of the crystal.  
$$R_{e} = R_{m} * (1 + \frac{C_{0}}{C_{L}})^{2}$$

<img src="-R_test.png" width="200"></img> 

3. Add a resistor Connect_r (or R<sub>Q</sub> in the image) directly in series with the crystal (this is not the resistor R_Ext at the output of the amplifier!).  Increase the resistor until the crystal stops oscillating.  Now it's possible to calculate the negative resistance of the crystal:

$$\left| -R_{n} \right|=Connect_R + R_{e}$$

## Oscillation allowance
$$\frac{\left| R_{n} \right|}{R_{e}} \ge  5$$

Fesz electronics formula divides by ESRmax instead of Re used in Seiko application note.

If the gain margin is too low, try a crystal that requires lower load capacitance.

# Oscillator transconductance

g<sub>m</sub> = actual transconductance of the amplifier, specified in the datasheet.  If the value is not specified, then measure it according to the method described in [Statek : Practical Analysis of the Pierce Oscillator](./tn31.pdf).

g<sub>mcrit</sub> is defined as the minimal transconductance required to maintain a stable oscillation.

$$g_{mcrit} =  4 \times (ESR+R_{Ext}) \times (2 \times π \times  f)^{2} \times (C_{0}+C_{L})^{2}$$

## Minimum required gain margin

$$gain_{margin} =  \frac{g_{m}}{g_{mcrit}} > 5$$

## Maximum allowed gain margin

Gm = maximal critical crystal transconductance, specified in the datasheet.

$$G_{m} \ge \frac{g_{m}}{5}$$

If the requirements are not met, try a crystal that requires lower load capacitance or a crystal with lower ESR.

# Maximum drive level
[Dr. Steve Arar : How to Measure the Drive Level of a Quartz Crystal](https://www.allaboutcircuits.com/technical-articles/measuring-drive-level-of-a-quartz-crystal/)

## Method 1 : Using a current probe
Good for you if you have an HF-current probe.

## Method 2 : Measuring the voltage at the input of the amplifier
The current through the crystal is the sum of the current into the amplifier and the current into external load capacitance C2.  We can ignore the current into the amplifier because it's very small compared to the current into the external load capacitance C2.

**An active FET-input probe (<1pF) is required to measure the voltage at the input of the amplifier.**

In [2]:
import math
# Calculate maximum drive level

Vpp = 0.632 # V, peak-to-peak, measured over C2, using a probe with capacitance << C2
f = 27.12e6 # Hz, frequency of the drive signal
ESR = 60.0 # Ohm, equivalent series resistance of the crystal
C2 = 18.0e-12 # F, second external load capacitance of the crystal
Cs = 5.0e-12 # F, stray capacitance of the crystal
Cprobe = 10.0e-12 # F, capacitance of the probe

# Calculate the load capacitance
CL = C2 + Cs/2 + Cprobe

# Calculate the drive level
P = (Vpp * math.pi * f * CL)**2 * ESR / 2
print("Maximum drive level: %.2f µW" % (P * 1e6))

Maximum drive level: 80.92 µW


## Method 3 : Measuring the differential voltage over the crystal
$$ P = \frac{R_{m}}{2} \times (\Pi^{} \times f \times (C_{0}+C_{L}) \times V_{pp})^{2} $$

In [19]:
import math
# Calculate maximum drive level, devttyS0

Vpp = 2 # V, peak to peak voltage across the crystal
C0 = 5.0e-12 # F, shunt capacitance
CL = 12.0e-12 # F, load capacitance
f = 10.0e6 # Hz, frequency of the drive signal
ESR= 50.0 # Ohm, equivalent series resistance of the crystal

Rm = ESR / (1 + C0 / CL)**2 # motional resistance of the crystal
P = Rm/2 * (math.pi*f*(C0+CL)*Vpp)**2 # power dissipated in the crystal

## Calculating the external resistor R_Ext

Reducing crystal drive level by adding a series resistor R_Ext on the output of the amplifier.
Start off with a value of :

In [20]:
C2 = 10.0e-12 # F, load capacitance on crystal on the amplifer side
f = 10.0e6 # Hz, frequency of the drive signal

R_Ext = 1 / (2 * math.pi * f * C2) # initial guess for the series resistance

Remark that some ICs already have a series resistor on the output.  You might want to measure the output impedance of the IC to get a more accurate value of Rs according to the [IQD application note](https://www.iqdfrequencyproducts.com/media/pg/1046/1551961314/31.pdf).

# References
* Good in-depth explanation of the Pierce oscillator : [Statek application notes](https://statek.com/technical-papers/)
* [Wikipedia Pierce oscillator](https://en.wikipedia.org/wiki/Pierce_oscillator)
* [devttys0 : Crystal Oscillator Theory and Design](https://youtu.be/5StwZCeNzVU?t=1398) : Maximum drive level
* [AN943 : Practical PICmicro® Oscillator Analysis and Design](http://ww1.microchip.com/downloads/en/AppNotes/00943A.pdf)
* [Seiko Oscillator Circuit Evaluation Method (1)](https://www5.epsondevice.com/en/information/technical_info/pdf/tech_notes_e201302.pdf)
* [Seiko Oscillator Circuit Evaluation Method (2)](https://www5.epsondevice.com/en/information/technical_info/pdf/tech_notes_e_oscillator_circuit_evaluation_method_2.pdf)
* [ST AN2867](https://www.st.com/resource/en/application_note/cd00221665-oscillator-design-guide-for-stm8af-al-s-stm32-mcus-and-mpus-stmicroelectronics.pdf)
* [IQD : Practical Analysis of the Pierce Oscillator](https://www.iqdfrequencyproducts.com/media/pg/1046/1551961314/31.pdf)